In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

from collections import Counter

import pickle
import torch
import torch.optim as optim
from allennlp.data.iterators import BasicIterator
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.token_embedders import Embedding
from allennlp.training.trainer import Trainer
from torch.nn import CosineSimilarity
from torch.nn import functional

In [2]:
from word2vec import SkipGramReader

In [ ]:
EMBEDDING_DIM = 256
BATCH_SIZE = 256
set_trace()

--Call--
> /Users/benoitpatra/Code/DeepLearningRealWorldNLP/venv/lib/python3.6/site-packages/IPython/core/displayhook.py(252)__call__()
    250         sys.stdout.flush()
    251 
--> 252     def __call__(self, result=None):
    253         """Printing with history cache management.
    254 

ipdb> EMBEDDING_DIM
*** NameError: name 'EMBEDDING_DIM' is not defined


In [10]:
class SkipGramModel(Model):
    def __init__(self, vocab, embedding_in):
        super().__init__(vocab)
        self.embedding_in = embedding_in
        self.linear = torch.nn.Linear(
            in_features=EMBEDDING_DIM,
            out_features=vocab.get_vocab_size('token_out'),
            bias=False)

    def forward(self, token_in, token_out):
        embedded_in = self.embedding_in(token_in)
        logits = self.linear(embedded_in)
        loss = functional.cross_entropy(logits, token_out)

        return {'loss': loss}

In [11]:
def get_related(token: str, embedding: Model, vocab: Vocabulary, num_synonyms: int = 10):
    """Given a token, return a list of top N most similar words to the token."""
    token_id = vocab.get_token_index(token, 'token_in')
    token_vec = embedding.weight[token_id]
    cosine = CosineSimilarity(dim=0)
    sims = Counter()

    for index, token in vocab.get_index_to_token_vocabulary('token_in').items():
        sim = cosine(token_vec, embedding.weight[index]).item()
        sims[token] = sim

    return sims.most_common(num_synonyms)

In [12]:
RAW_FILE='./inputs/text8'
PICKLE='./outputs/skipgramtext8_top1e5.pickle'

In [13]:
reader = SkipGramReader(kept_tokens=int(1e5))
text8 = reader.read(RAW_FILE)
with open(PICKLE, 'wb') as handle:
    pickle.dump(text8, handle, protocol=pickle.HIGHEST_PROTOCOL)

0it [00:00, ?it/s]Your label namespace was 'token_in'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
Your label namespace was 'token_out'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
8311it [00:02,  1.35s/it]

Total tokens length: 17005207
Kept tokens length: 100000


999970it [00:09, 102628.69it/s]


with open(PICKLE, 'rb') as handle:
    text8 = pickle.load(handle)

In [14]:
print(text8[:20])

[<allennlp.data.instance.Instance object at 0x1255bf0f0>, <allennlp.data.instance.Instance object at 0x1251104a8>, <allennlp.data.instance.Instance object at 0x1358b8240>, <allennlp.data.instance.Instance object at 0x142e5d1d0>, <allennlp.data.instance.Instance object at 0x142e5d240>, <allennlp.data.instance.Instance object at 0x142e5d2e8>, <allennlp.data.instance.Instance object at 0x142e5d358>, <allennlp.data.instance.Instance object at 0x142e5d3c8>, <allennlp.data.instance.Instance object at 0x142e5d438>, <allennlp.data.instance.Instance object at 0x142e5d4a8>, <allennlp.data.instance.Instance object at 0x142e5d518>, <allennlp.data.instance.Instance object at 0x142e5d5c0>, <allennlp.data.instance.Instance object at 0x142e5d630>, <allennlp.data.instance.Instance object at 0x142e5d6a0>, <allennlp.data.instance.Instance object at 0x142e5d710>, <allennlp.data.instance.Instance object at 0x142e5d780>, <allennlp.data.instance.Instance object at 0x142e5d7f0>, <allennlp.data.instance.Instan

In [15]:
vocab = Vocabulary.from_instances(
    text8, min_count={'token_in': 5, 'token_out': 5})

100%|██████████| 999970/999970 [00:01<00:00, 595929.94it/s]


In [16]:
iterator = BasicIterator(batch_size=BATCH_SIZE)
iterator.index_with(vocab)

In [17]:
embedding_in = Embedding(num_embeddings=vocab.get_vocab_size('token_in'),
                         embedding_dim=EMBEDDING_DIM)

In [18]:
model = SkipGramModel(vocab=vocab,
                      embedding_in=embedding_in)
optimizer = optim.Adam(model.parameters())
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=text8,
                  num_epochs=5, serialization_dir='./outputs')
trainer.train()

loss: 7.1469 ||: 100%|██████████| 3907/3907 [06:05<00:00, 10.69it/s]
loss: 6.7890 ||: 100%|██████████| 3907/3907 [06:04<00:00, 10.72it/s]
loss: 6.5293 ||: 100%|██████████| 3907/3907 [06:03<00:00, 10.75it/s]
loss: 6.2738 ||: 100%|██████████| 3907/3907 [06:03<00:00, 10.75it/s]
loss: 6.0563 ||: 100%|██████████| 3907/3907 [06:02<00:00, 10.78it/s]


{'best_epoch': 4,
 'peak_cpu_memory_MB': 1457.102848,
 'training_duration': '0:30:19.891687',
 'training_start_epoch': 0,
 'training_epochs': 4,
 'epoch': 4,
 'training_loss': 6.056346551701331,
 'training_cpu_memory_MB': 1457.102848}

In [19]:
print(get_related('one', embedding_in, vocab))

[('one', 1.0), ('nine', 0.6574505567550659), ('six', 0.6249205470085144), ('breed', 0.6217449903488159), ('restless', 0.6176639795303345), ('hardcover', 0.6125352382659912), ('garter', 0.6064285039901733), ('lode', 0.5996147990226746), ('seattle', 0.5982744097709656), ('eight', 0.5981407761573792)]


In [20]:
print(get_related('december', embedding_in, vocab))

[('december', 1.0), ('yielding', 0.6932260394096375), ('seattle', 0.6855131983757019), ('runner', 0.6828048825263977), ('crafoord', 0.6820924282073975), ('dienne', 0.6811384558677673), ('duke', 0.6755971908569336), ('vermont', 0.6632827520370483), ('makhnovschina', 0.6550191640853882), ('exhausted', 0.6542066931724548)]


In [21]:
set_trace()

--Call--
> /Users/benoitpatra/Code/DeepLearningRealWorldNLP/venv/lib/python3.6/site-packages/IPython/core/displayhook.py(252)__call__()
    250         sys.stdout.flush()
    251 
--> 252     def __call__(self, result=None):
    253         """Printing with history cache management.
    254 

ipdb> text8
*** NameError: name 'text8' is not defined
ipdb> 
*** NameError: name 'text8' is not defined
ipdb> 
*** NameError: name 'text8' is not defined
ipdb> exit()


BdbQuit: 

In [ ]:
a = 4;
import pdb; pdb.set_trace();